In [416]:
import spacy
import pandas as pd
from spacy.matcher import PhraseMatcher
from spacy.tokens import Doc

In [376]:
df = pd.read_csv("C:/Users/dschw/Documents/GitHub/Thesis/Output/NCCR_combined_corpus_DE_wording_available_TEST.csv")

In [394]:
nlp = spacy.load("de_core_news_lg", exclude=['tok2vec', 'tagger', 'morphologizer', 'parser',
                         'attribute_ruler', 'lemmatizer'])

nlp.add_pipe("sentencizer")



In [395]:
def standardize_text(text):
    # Replace special characters #todo: simplify replacement
    text = text.replace("ä", "ae").replace("ü", "ue").replace("ö", "oe").replace("Ö", "Oe").replace("Ä", "Ae").replace("Ü", "Ue").replace("ß", "ss").replace("@", "").replace("/", "")
    text = text.replace("ç", "c"). replace("", "")
    
    text = text.lower()
    text = " ".join(text.split())

    return text

In [396]:
def standardize_wording(wording):
    # Replace special characters #todo: simplify replacement
    wording = wording.replace("/", "").replace("@", "").replace("<ord:65430>", "")
    wording = " ".join(wording.split())

    return wording

In [397]:
df['text_prep'] = df['text_prep'].apply(lambda x: standardize_text(x))

In [398]:
df['Wording'] = df['Wording'].apply(lambda x: standardize_wording(x))

In [400]:
df['text_prep'] 

0      "bucher fordert ehrenerklaerung von allen parl...
1      na berlin (ots) - der rheinland-pfaelzische fi...
2      wallner: gruene transparenz-offensive soll kla...
3      zum geheimdienstskandal weltweit umfassender a...
4      "genug gezahlt - ueberstunden steuerfrei", "jo...
                             ...                        
944    na berlin (ots) - buendnis 90die gruenen haben...
945    na berlin. vor der parlamentswahl in simbabwe ...
946    na muenchen die 600 plaetze der festhalle des ...
947    bilaterale ja eu-beitritt nein. dank den bilat...
948    na bemaengelte oekonomisierung von ehe und fam...
Name: text_prep, Length: 949, dtype: object

In [401]:
df['Wording'] 

0      bucher fordert ehrenerklaerung von allen parla...
1      sein parteikollege, der rheinland-pfaelzische ...
2      das team stronach hat bis april 2013 laut rech...
3                                                    wir
4      waehrend andere parteien mit nackter haut vers...
                             ...                        
944                                              gruenen
945    die fdp-bundestagsfraktion fordert alle staatl...
946                                      die altparteien
947                                                  svp
948                                        norbert bluem
Name: Wording, Length: 949, dtype: object

In [425]:
df['doc'] = list(nlp.pipe(df['text_prep']))
df['wording_doc'] = list(nlp.pipe(df['Wording']))




In [424]:
doc = nlp('Das ist ein Test der das Ganze prüft')
get_sub_wording(doc)

Das ist ein Test der 

In [436]:
def get_sub_wording(wording):
    doc2 = Doc(wording.vocab, words=[t.text for i, t in enumerate(wording) if i <3])
    return doc2 
    


In [437]:
df['wording_doc'] = df['wording_doc'].apply(lambda x: get_sub_wording(x))

In [438]:
df['wording_doc']

0          (bucher, fordert, ehrenerklaerung)
1                    (sein, parteikollege, ,)
2                       (das, team, stronach)
3                                       (wir)
4                (waehrend, andere, parteien)
                        ...                  
944                                 (gruenen)
945    (die, fdp-bundestagsfraktion, fordert)
946                        (die, altparteien)
947                                     (svp)
948                          (norbert, bluem)
Name: wording_doc, Length: 949, dtype: object

In [439]:
# Define function to find index of tokens that match Wording content
def get_matches(doc, wording):
    
    # Define Spacy Matcher
    matcher = PhraseMatcher(nlp.vocab)
    # Add patterns from nlp-preprocessed Wording column
    matcher.add("WORDING", [wording])
    
    # Get matches
    matches = matcher(doc)

    return matches



In [440]:
df['matches_new'] = df.apply(lambda x: get_matches(x['doc'], x['wording_doc']), axis=1)

## Fix non-matches

In [441]:
df['matches_new']

0                          [(1212883764967918647, 1, 4)]
1                      [(1212883764967918647, 345, 348)]
2      [(1212883764967918647, 791, 794), (12128837649...
3      [(1212883764967918647, 29, 30), (1212883764967...
4                        [(1212883764967918647, 60, 63)]
                             ...                        
944    [(1212883764967918647, 8, 9), (121288376496791...
945                      [(1212883764967918647, 68, 71)]
946                    [(1212883764967918647, 117, 119)]
947                      [(1212883764967918647, 97, 98)]
948                      [(1212883764967918647, 19, 21)]
Name: matches_new, Length: 949, dtype: object

In [442]:
non = df[~df['matches_new'].astype(bool)]
non = non[['ID', 'doc', 'Wording', 'matches_new', 'wording_segments']]
#non['wording_doc'] = list(nlp.pipe(non['Wording']))
non['doc_tokens'] = non['doc'].apply(lambda x: [token.text for token in x])
non['wording_tokens'] = non['wording_doc'].apply(lambda x: [token.text for token in x])

In [443]:
len(non)

47

In [444]:
non['Wording'].iloc[1]

'volksparte'

In [446]:
non['doc'].iloc[1]

1.6. oesterreich und europa. europaeische union. die volkspartei sieht in der zukuenftigen mitgliedschaft oesterreichs zur europaeischen union die wesentlichste voraussetzung zum ausbau unserer, wirtschaftskraft und zur sicherung der beschaeftigung in oesterreich. der zielstrebigkeit der volkspartei ist es zu verdanken, dass oesterreich am 17. juli 1989 den antrag auf beitritt zu den europaeischen gemeinschaften gestellt hat. am 12. juni 1994 haben die oesterreicherinnen und oesterreicher mit ueberwaeltigender mehrheit entschieden, dass oesterreich mit beginn des kommenden jahres der europaeischen union als gleichberechtigter partner angehoeren wird. damit wurde fuer oesterreich die wichtigste entscheidung seit dem abschluss des staatsvertrag zum wohle unserer heimat getroffen. die antwort der europaeischen union auf die weltweiten herausforderungen, vor allem im technischen und wirtschaftlichen bereich, sind der binnenmarkt und die einheitliche europaeische akte sowie deren weiterentw

In [447]:
print(str(non.iloc[2]['doc_tokens']))

['liberal-sozial', ':-)']


In [448]:
print(str(non.iloc[2]['wording_tokens']))

['barbara', 'schmid-federer']


In [ ]:
--> try match first 5 tokens

## Investigate Missing Segments

In [100]:
# sample type?
import pandas as pd
non_train = pd.read_csv("C:/Users/dschw/Documents/GitHub/Thesis/Output/non_matched_TRAIN.csv")
non_test = pd.read_csv("C:/Users/dschw/Documents/GitHub/Thesis/Output/non_matched_TEST.csv")

In [101]:
non_train

,Unnamed: 0,level_0,index,ID,text,Coder,#TS,#TS2,Codierstart_Lab,Medium,...,year,text_temp,Wording_temp,doc_temp,Wording_doc_temp,wording_matches,wording_sentence,wording_segments,doc_tokens,wording_tokens
0,1734,1733,856,au_pr_el_13_050024.txt,Text-ID: at_pr_el_13_050024.txt\n\nPress Relea...,difr,1.430399e+09,1.430400e+09,Thu Apr 30 15:00:42 2015,1150,...,2013,Euro belastete. Angesichts dieser doch unerwar...,"SPOe-Frauenministerin Heinisch-Hosek,",Euro belastete. Angesichts dieser doch unerwar...,"SPOe-Frauenministerin Heinisch-Hosek ,",[],[],NaN,"['Euro', 'belastete', '.', 'Angesichts', 'dies...","['SPOe-Frauenministerin', 'Heinisch-Hosek', ',']"
1,3658,3658,2693,de_fb_db_15061_00120.txt,Text ID:\nde_fb_db_15061_00120.txt\n\n\nFacebo...,stbu,1.439121e+09,1.439127e+09,Sun Aug 09 13:47:54 2015,9902,...,2014,"Industrie pauschal befreien, VerbraucherInnen ...",Sigmar Gabriel will die Industrie weiterhin fa...,"Industrie pauschal befreien, VerbraucherInnen ...",Sigmar Gabriel will die Industrie,[],[],NaN,"['Industrie', 'pauschal', 'befreien', ',', 'Ve...","['Sigmar', 'Gabriel', 'will', 'die', 'Industrie']"
2,2775,2775,1449,cd_fb_db_13332_00018_2.txt,Text ID:\ncd_fb_db_13332_00018.txt\n\n\nFacebo...,lamo,1.470725e+09,1.470725e+09,Tue Aug 09 08:38:54 2016,9902,...,2015,Startschuss zum Abstimmungskampf der Aktion Me...,SVP-Nationalrat Gregor Rutz (ZH),Startschuss zum Abstimmungskampf der Aktion Me...,SVP-Nationalrat Gregor Rutz ( ZH,[],[],NaN,"['Startschuss', 'zum', 'Abstimmungskampf', 'de...","['SVP-Nationalrat', 'Gregor', 'Rutz', '(', 'ZH']"
3,5203,5203,3478,de_pr_el_13_041174.txt,Text-ID: de_pr_el_13_041174.txt\n\nPress Relea...,chfi,1.429732e+09,1.429732e+09,Wed Apr 22 21:38:31 2015,1550,...,2013,Zur CDU-internen Auseinandersetzung ueber die ...,CDUCSU-,Zur CDU-internen Auseinandersetzung ueber die ...,CDUCSU-,[],[],NaN,"['Zur', 'CDU-internen', 'Auseinandersetzung', ...",['CDUCSU-']
4,3543,3539,2490,de_fb_db_15039_00185.txt,Text ID:\nde_fb_db_15039_00185.txt\n\n\nFacebo...,cidu,1.456489e+09,1.456489e+09,Fri Feb 26 13:08:35 2016,9902,...,2014,"Was der Mensch braucht: Modernisierte Schulen,...",Die US-Regierung forciert Medienberichten zufo...,"Was der Mensch braucht: Modernisierte Schulen,...",Die US-Regierung forciert Medienberichten zufo...,[],[],NaN,"['Was', 'der', 'Mensch', 'braucht', ':', 'Mode...","['Die', 'US-Regierung', 'forciert', 'Medienber..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,3483,3483,2379,de_fb_db_15007_00027z.txt,Text ID:\nde_fb_db_15007_00027.txt\n\n\nFacebo...,mapr,1.459284e+09,1.459285e+09,Tue Mar 29 22:40:17 2016,9902,...,2015,Victor #Orbàn ist heute zu Gast bei der #CSU. ...,Orban,Victor #Orbàn ist heute zu Gast bei der #CSU. ...,Orban,[],[],NaN,"['Victor', '#', 'Orbàn', 'ist', 'heute', 'zu',...",['Orban']
213,3482,3482,2376,de_fb_db_15007_00023z.txt,Text ID:\nde_fb_db_15007_00023.txt\n\n\nFacebo...,mabu,1.464203e+09,1.464205e+09,Wed May 25 21:01:00 2016,9902,...,2015,#Asyl Das Grundgesetz ist die gemeinsame Grund...,Wir brauchen mehr Anpackgesellschaft<ORD:65427>.,#Asyl Das Grundgesetz ist die gemeinsame Grund...,Wir brauchen mehr Anpackgesellschaft <,[],[],NaN,"['#', 'Asyl', 'Das', 'Grundgesetz', 'ist', 'di...","['Wir', 'brauchen', 'mehr', 'Anpackgesellschaf..."
214,2583,2583,1144,au_pr_el_13_111082.txt,Text-ID: at_pr_el_13_111082.txt\n\nPress Relea...,pagr,1.424854e+09,1.424855e+09,Wed Feb 25 09:46:41 2015,1150,...,2013,Hinweisen auf Vorfaelle im Apotheken-Bereich m...,Datenmissbrauch-Skandal: Lugar fordert soforti...,Hinweisen auf Vorfaelle im Apotheken-Bereich m...,Datenmissbrauch-Skandal : Lugar fordert sofort...,[],[],NaN,"['Hinweisen', 'auf', 'Vorfaelle', 'im', 'Apoth...","['Datenmissbrauch-Skandal', ':', 'Lugar', 'for..."
215,983,983,599,au_pr_el_13_020021.txt,Text-ID: at_pr_el_13_020021.txt\n\nPress Relea...,pagr,1.424853e+09,1.424854e+09,Wed Feb 25 09:38:09 2015,1150,...,2013,"Zeit fuer einen Kanzlerwechsel Wien, 27. Septe...",Michael Spind

In [102]:
list(non_train)

['Unnamed: 0',
 'level_0',
 'index',
 'ID',
 'text',
 'Coder',
 '#TS',
 '#TS2',
 'Codierstart_Lab',
 'Medium',
 'Author',
 'Author_ID',
 'Date',
 'Length',
 'TS_Beginning',
 'TS_End',
 'Images',
 'Position',
 'Genre',
 'Main_Issue',
 'Frame_HI_face',
 'Frame_HI_indi',
 'Frame_HI_emoN',
 'Frame_HI_emoP',
 'Frame_Con_disa',
 'Frame_Con_repr',
 'Frame_Con_winn',
 'Frame_Con_slant',
 'Frame_PRF_conduit',
 'Frame_PRF_context',
 'Frame_PRF_interp',
 'Frame_PRF_account',
 'Frame_Neg_tone',
 'Frame_Neg_scep',
 'Frame_Neg_other',
 'Bemerkungen',
 'Count_Speaker',
 'Count_Issues',
 'Count_ActEval',
 'STYLE_Negativ',
 'STYLE_Emot',
 'T_Brutto',
 'T_Break',
 'T_Netto',
 'T_H',
 'Sessiontext',
 'Session_ID',
 'Sample',
 'Sample_Country',
 'Sample_Lang',
 'Sample_Type',
 'Weight',
 'Filter_Coded',
 'STYLE_Colloquial',
 'STYLE_Casual',
 'STYLE_Facts',
 'STYLE_Sense',
 'STYLE_BlackWhite',
 'STYLE_Sarcasm',
 'STYLE_Drama',
 'STYLE_EmoTone',
 'STYLE_CommMan',
 'STYLE_UsThem',
 'STYLE_Privat',
 'STYLE_Co

In [103]:
non_train.iloc[3]['doc_tokens']

"['Zur', 'CDU-internen', 'Auseinandersetzung', 'ueber', 'die', 'kuenftige', 'Ruestungsexportpolitik', 'erklaert', 'Katja', 'Keul', ',', 'Parlamentarische', 'Geschaeftsfuehrerin', ':', 'Die', 'Abgrenzung', 'der', 'CDU-Fraktionsfuehrung', 'von', 'den', 'Vorstellungen', 'der', 'eigenen', 'Verteidigungspolitiker', 'zur', 'Ruestungsexportpolitik', 'ist', 'nicht', 'glaubwuerdig', '.', 'Die', 'schwarz-gelbe', 'Bundesregierung', 'hat', 'in', 'den', 'vergangenen', 'vier', 'Jahren', 'die', 'Ruestungsexportrichtlinien', 'ausgehoehlt', '.', 'Die', 'Bundesregierung', 'hat', 'diese', 'Grundsaetze', 'ein', 'ums', 'andere', 'Mal', 'verletzt', ',', 'als', 'sie', 'Waffenlieferungen', 'in', 'Konfliktregionen', 'und', 'menschenrechtsverletzende', 'Staaten', 'wie', 'Saudi-Arabien', 'genehmigt', 'hat', '.', 'Wenn', 'die', 'CDUCSU-Fraktion', 'nun', 'betont', ',', 'an', 'dieser', 'Praxis', 'festhalten', 'zu', 'wollen', ',', 'bedeutet', 'das', ',', 'dass', 'Deutschland', 'bei', 'einer', 'CDU-gefuehrten', 'Regi

In [104]:
non_train.iloc[3]['wording_tokens']
#non_train.iloc[221]['Wording']

"['CDUCSU-']"

In [105]:
for index, row in non_train.iterrows():
    print(row['doc_tokens'])
    print(row['wording_tokens'])
    print("___")

['Euro', 'belastete', '.', 'Angesichts', 'dieser', 'doch', 'unerwartet', 'hohen', 'Kosten', '-', 'gerade', 'die', 'Frauenministerin', 'betreffend', '-', 'kuendigte', 'der', 'FPOe-Abgeordnete', 'Doppler', 'an', ',', 'mittels', 'weiterer', 'parlamentarischer', 'Anfragen', ',', 'die', 'tatsaechlichen', 'Leistungen', ',', 'die', 'Vergabe', 'der', 'Auftraege', ',', 'sowie', 'die', 'genauen', 'Inhalte', 'dieser', 'Coachings', 'pruefen', 'zu', 'wollen', '.']
['SPOe-Frauenministerin', 'Heinisch-Hosek', ',']
___
['Industrie', 'pauschal', 'befreien', ',', 'VerbraucherInnen', 'belasten', ',', 'Erneuerbare', 'ausbremsen', '.', 'So', 'geht', '#', 'GroKo-#Energiewende-Wende', '.', 'http', ':', 't.coFTYHbkTHmP']
['Sigmar', 'Gabriel', 'will', 'die', 'Industrie']
___
['Startschuss', 'zum', 'Abstimmungskampf', 'der', 'Aktion', 'Medienfreiheit', ':', 'Nein', 'zur', 'RTVG-Revision', 'am', '14.', 'Juni', '!', 'http', ':', 'www.blick.chnewspolitikes-schnaubt-vor-wut-dieses-emoji-soll-die-billag-bodigen-id37

In [32]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(non_train.Sample_Type)

0        SocialMedia
1       PressRelease
2        SocialMedia
3        SocialMedia
4       PressRelease
5       PressRelease
6        SocialMedia
7        SocialMedia
8        SocialMedia
9        SocialMedia
10      PressRelease
11     Past_PartyMan
12       SocialMedia
13      PressRelease
14       SocialMedia
15      PressRelease
16      PressRelease
17      PressRelease
18       SocialMedia
19          PartyMan
20       SocialMedia
21      PressRelease
22      PressRelease
23      PressRelease
24      PressRelease
25       SocialMedia
26      PressRelease
27      PressRelease
28          PartyMan
29     Past_PartyMan
30      PressRelease
31      PressRelease
32      PressRelease
33          PartyMan
34       SocialMedia
35     Past_PartyMan
36       SocialMedia
37       SocialMedia
38      PressRelease
39          PartyMan
40     Past_PartyMan
41      PressRelease
42      PressRelease
43      PressRelease
44     Past_PartyMan
45      PressRelease
46          PartyMan
47       Soci